# Plotting all confusion matrices - supplementary figures

Code for making figures A11 - A17

In [ ]:
## Import packages

script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using CSV
using JLD
using DataFrames
using JSON
using MAT
using Plots

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)

In [ ]:
### Set parameters

betti_colors = [["#243a4c"] ["#406372"] ["#66939e"] ["#9bc3c6"]]

# Read from config file
config_file = "config101220.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("_bettis",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING, x), betti_files)
betti_files = filter(x->!occursin("dsi", x), betti_files)

println("Located the following graph files:")
# for betti_file in betti_files
#     println(betti_file)
# end


# Locate the nametags - a nametag here is the run type. For example 'forward','thresh_rho07_threshold', etc.
nametags = []
for betti_file in betti_files
    println(betti_file)
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettis")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)


for nametag in nametags
    namefiles = filter(x->occursin(nametag,x), betti_files)
    if length(namefiles)<12
    println(nametag)
    println(length(namefiles))
    end
end

nametags

In [ ]:
## Filter to include JUST the threshold files of the main models
thresh_files = filter(x -> occursin("threshold",x),betti_files)
thresh_files = filter(x -> !occursin("randomized",x),thresh_files)
thresh_files = filter(x -> !occursin("clique",x),thresh_files)
thresh_files = filter(x -> !occursin("Triangle",x),thresh_files)
thresh_files = filter(x -> !occursin("noise",x),thresh_files)

In [ ]:
# Set model names to make loading and plotting easier.

model_names = ["IID","assortative","coreperiphery", "cosineGeometric","disassortative", "discreteUniform",
    "dotProduct", "geometricConf", "randomGeometric", "ringLattice", "rmsd", "squaredEuclidean" ]
mat_dir = "../processed_data/classification/"
mat_files = readdir(mat_dir)
mat_files = filter(x -> occursin(".mat",x), mat_files)
mat1 = matread("$(mat_dir)/$(mat_files[1])")
mat_files

In [ ]:
# Create a list of classification (.mat) types. In the text, 'prenoise' is called 'model weights' and 'blues'
# is called 'noise-exclusive'

mat_types = []
for mat_file in mat_files
    mat_types = [mat_types; split(mat_file,"_")[2]]
end
unique!(mat_types)

In [ ]:
# Loop over mat types and create plots

# Colors
correct = "#03524C"
incorrect = "#531B56"


# Loop through mat files
mat_file_types = filter(x -> occursin(mat_types[1],x), mat_files)
println(mat_file_types)
# multiply all the off diagonal entries by -1 so that the diverging colormap works appropriately
thresholds = collect(0.1:0.05:0.9)

for mat_type in mat_types
    
    plot_array = Any[]
    
    for threshold in thresholds

        mat1 = matread("$(mat_dir)/matrix_$(mat_type)_$threshold.mat")

        factor = (-1).*ones(size(mat1["C"])) .+ (2 .*Matrix(I,size(mat1["C"])))

        if threshold==0.9


            p_i= heatmap(mat1["C"].*factor, c=cgrad([incorrect, :white, correct], [0,0.5, 1]),
                clim=(-250,250), yflip=true,
                xrotation=90,
                fontfamily=:Times, aspect_ratio=:equal, grid=false)
            xticks!(collect(1:12), model_names)
            yticks!(collect(1:12), model_names)

        else
            p_i= heatmap(mat1["C"].*factor, c=cgrad([incorrect, :white, correct], [0,0.5, 1]),
                clim=(-250,250), yflip=true,
                xrotation=90,
                fontfamily=:Times, aspect_ratio=:equal, axis=nothing, legend=nothing, border=:none)
            xticks!(collect(1:12), model_names)
            yticks!(collect(1:12), model_names)
            title!("rho_t = $(threshold)")
        end

        plot(p_i,size=(300,300))
        push!(plot_array,p_i)

    end

    l = @layout [a b c d; e f g h; i j k l; m n o p; q]
    plot(plot_array..., layout=l, size=(3000,4000))
    savefig("../Figures/confusion_matrices_$(mat_type)_all.pdf")
end